<a href="https://colab.research.google.com/github/aisancesg/mixed/blob/main/sber_task_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Есть таблица с данными по кредитным линиям юридических лиц

Для нас важны столбцы:
- Отчётная дата
- ID договора
- Дата окончания периода доступности кредитного лимита
- Дата окончательного погашения кредитной линии
    
Требуется:
1. Отфильтровать из таблицы сделки, для которых период доступности кредитного лимита изменяется;
2. Для тех сделок, у которых он отсутствует, проставить его самой первой датой окончательного погашения для каждой кредитной линии.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Для задачи по Python.csv', sep = ';')
for column in ['ReportDate', 'DrawPeriodEndDate', 'MaturityDate'] :
    df[column] = pd.to_datetime(df[column])
print(df.info(verbose = True))
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ReportDate         6 non-null      datetime64[ns]
 1   AgreementID        6 non-null      int64         
 2   DrawPeriodEndDate  4 non-null      datetime64[ns]
 3   MaturityDate       6 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1)
memory usage: 324.0 bytes
None


C:\Users\User\AppData\Local\Temp\ipykernel_17532\1839212241.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column])
C:\Users\User\AppData\Local\Temp\ipykernel_17532\1839212241.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column])
C:\Users\User\AppData\Local\Temp\ipykernel_17532\1839212241.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column])


,ReportDate,AgreementID,DrawPeriodEndDate,MaturityDate
0,2020-01-31,1,2020-06-30,2020-12-31
1,2020-02-29,1,2020-12-31,2020-12-31
2,2020-01-31,2,NaT,2020-11-30
3,2020-02-29,2,NaT,2020-12-31
4,2020-01-31,3,2020-08-31,2020-12-31
5,2020-02-29,3,2020-08-31,2020-12-31


**1. Удалить сделки с меняющимся признаком DrawPeriodEndDate**

In [ ]:
condition=df.groupby('AgreementID', as_index=False)\
    .nunique()\
    .rename(columns={'DrawPeriodEndDate' :'count_unique_DrawPeriodEndDate'})\
    .query('count_unique_DrawPeriodEndDate < 2')
df_1=df[df.AgreementID.isin(condition.AgreementID)]
df_1

,ReportDate,AgreementID,DrawPeriodEndDate,MaturityDate
2,2020-01-31,2,NaT,2020-11-30
3,2020-02-29,2,NaT,2020-12-31
4,2020-01-31,3,2020-08-31,2020-12-31
5,2020-02-29,3,2020-08-31,2020-12-31


**2. Для сделок с отсутствующим признаком DrawPeriodEndDate установить для него первое по времени значение MaturityDate**

In [ ]:
df_1.loc[df_1['DrawPeriodEndDate'].isnull(), 'DrawPeriodEndDate'] = df_1.groupby('AgreementID')['MaturityDate'].transform('min')

In [ ]:
df_1

,ReportDate,AgreementID,DrawPeriodEndDate,MaturityDate
2,2020-01-31,2,2020-11-30,2020-11-30
3,2020-02-29,2,2020-11-30,2020-12-31
4,2020-01-31,3,2020-08-31,2020-12-31
5,2020-02-29,3,2020-08-31,2020-12-31
